In [7]:
!gcloud auth application-default revoke

You are about to revoke the credentials stored in: 
[/Users/andrewcooley/.config/gcloud/application_default_credentials.json]

Do you want to continue (Y/n)?  ^C


Command killed by keyboard interrupt



In [6]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=mtIeOleAnDsAM6ModPOkWTwCEftZ8v&access_type=offline&code_challenge=7pP_B2A2Wl8YZa2Jf3B4qmfQAI3Oah2nIwiYPIJpvGo&code_challenge_method=S256


Credentials saved to file: [/Users/andrewcooley/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "andrewcooley-test-project" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabl

In [5]:
!gcloud auth application-default set-quota-project "andrewcooley-genai-tests"

ERROR: (gcloud.auth.application-default.set-quota-project) User [andrewcooley@google.com] does not have permission to access projects instance [andrewcooley-genai-tests:testIamPermissions] (or it may not exist): Caller does not have required permission to use project andrewcooley-genai-tests. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=andrewcooley-genai-tests and then retry. Propagation of the new permission may take a few minutes.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developer console IAM admin
    url: https://console.developers.google.com/iam-admin/iam/project?project=andrewcooley-genai-tests
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googleapis.com
  metadata:
    consumer: projects/andrewcooley-genai-tests
    service: cloudresourcemanager.googleapis.com


In [8]:
from google.cloud import aiplatform

PROJECT_ID = "andrewcooley-genai-tests"
REGION_1 = "us-east1"
EXPERIMENT_1 = "exp-test-1"
REGION_2 = "us-east4"
EXPERIMENT_2 = "exp-test-2"

aiplatform.init(project=PROJECT_ID, location=REGION_1, experiment=EXPERIMENT_1)

Creating Tensorboard


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard


Create Tensorboard backing LRO: projects/511623653677/locations/us-east1/tensorboards/3112972504931500032/operations/7606149586704400384


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/511623653677/locations/us-east1/tensorboards/3112972504931500032/operations/7606149586704400384


Tensorboard created. Resource name: projects/511623653677/locations/us-east1/tensorboards/3112972504931500032


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Tensorboard created. Resource name: projects/511623653677/locations/us-east1/tensorboards/3112972504931500032


To use this Tensorboard in another session:


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:


tb = aiplatform.Tensorboard('projects/511623653677/locations/us-east1/tensorboards/3112972504931500032')


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/511623653677/locations/us-east1/tensorboards/3112972504931500032')


In [2]:
PROJECT_ID = "andrewcooley-genai-tests"
EXPERIMENT_NAME = 'sdk-experiment'
EXPERIMENT_DESCRIPTION = 'metadata default creation test'
REGION = 'us-central1'

In [3]:
PROJECT_ID = "andrewcooley-genai-tests"
LOCATION = "us-central1"


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
# General
import inspect
from uuid import uuid4
from IPython.display import display, Markdown, HTML
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
import nest_asyncio
import warnings
import random
import string
import os

# Main
import vertexai
from vertexai.preview.evaluation import (
    EvalTask,
    PromptTemplate,
    CustomMetric,
    make_metric,
)
import pandas as pd
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel
from vertexai.generative_models import GenerativeModel, HarmCategory, HarmBlockThreshold

In [5]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [7]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specifed length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown(f"### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["instruction", "context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"{col}: {row[col]}"))
        display(HTML(""))


def plot_radar_plot(eval_results, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )

    fig.show()


def plot_bar_plot(eval_results, metrics=None):
    fig = go.Figure()
    data = []

    for eval_result in eval_results:
        title, summary_metrics, _ = eval_result
        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()


def print_aggregated_metrics(job):
    """Print AutoMetrics"""

    rougeLSum = round(job.rougeLSum, 3) * 100
    display(
        HTML(
            f"The {rougeLSum}% of the reference summary is represented by LLM when considering the longest common subsequence (LCS) of words."
        )
    )


def print_autosxs_judgments(df, n=3):
    """Print AutoSxS judgments in the notebook"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)

    for index, row in df.iterrows():
        if row["confidence"] >= 0.5:
            display(
                HTML(
                    f"Document: {row['id_columns']['document']}"
                )
            )
            display(
                HTML(
                    f"Response A: {row['response_a']}"
                )
            )
            display(
                HTML(
                    f"Response B: {row['response_b']}"
                )
            )
            display(
                HTML(
                    f"Explanation: {row['explanation']}"
                )
            )
            display(
                HTML(
                    f"Confidence score: {row['confidence']}"
                )
            )
            display(HTML(""))


def print_autosxs_win_metrics(scores):
    """Print AutoSxS aggregated metrics"""

    score_b = round(scores["autosxs_model_b_win_rate"] * 100)
    display(
        HTML(
            f"AutoSxS Autorater prefers {score_b}% of time Model B over Model A "
        )
    )

In [8]:

instructions = [
    "What commonly inspires individuals to pursue their current career paths?",
    "In general, how do professionals approach problem-solving in their daily work?",
    "Can you provide an example of a significant challenge that professionals often face and the common lessons learned?",
    "What typically motivates individuals to continually improve and learn new things in their respective fields?",
    "How do professionals commonly handle stress and manage tight deadlines?",
    "Can you describe a project or accomplishment that is often considered noteworthy in various fields?",
    "What aspects of work are generally found to be most fulfilling across professions?",
]

eval_dataset = pd.DataFrame(
    {
        "content": instructions,
    }
)

qa_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=["safety", "text_generation_quality"],
    experiment="qa-eval-01",
)

gemini_model_for_qa = GenerativeModel(
    "gemini-pro",
    generation_config={
        "temperature": 0.8,
        "top_k": 1,
    },
)

result = qa_eval_task.evaluate(
    model=gemini_model_for_qa, experiment_run_name=f"gemini-pro-for-qa-{uuid4()}"
)

PermissionDenied: 403 Permission 'aiplatform.metadataStores.get' denied on resource '//aiplatform.googleapis.com/projects/andrewcooley-genai-tests/locations/us-central1/metadataStores/default' (or it may not exist). [reason: "IAM_PERMISSION_DENIED"
domain: "aiplatform.googleapis.com"
metadata {
  key: "resource"
  value: "projects/andrewcooley-genai-tests/locations/us-central1/metadataStores/default"
}
metadata {
  key: "permission"
  value: "aiplatform.metadataStores.get"
}
]